# Data preparation for calibration benchmarks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from copy import deepcopy, copy
import os

from ctapipe.utils import get_dataset_path
from ctapipe.io import event_source
from ctapipe.calib import CameraCalibrator
from ctapipe.visualization import CameraDisplay
from ctapipe.io import HDF5TableWriter
from ctapipe.io.containers import Container, Field
from ctabench import tel_to_impact_point_distance

In [ ]:
infile = '/Users/thomasvuillaume/Work/CTA/Data/simu_mc/Simtel/gamma_40deg_0deg_run102___cta-prod3-lapalma3-2147m-LaPalma_cone10.simtel.gz'
max_events = None
outdir = '.'

integrators = [
    'NeighbourPeakIntegrator',
    'LocalPeakIntegrator',
    'FullIntegrator',
]

In [ ]:
class Image(Container):
    calibrated = Field(None)
    true_pe = Field(None)
    
class DL1_parameters(Container):
    mc_distance_tel_impact = Field(float, 'Distance from the telescope to the impact point')
    
def distance_tel_impact(event, tel_id):
    tel_pos = event.inst.subarray.tel_coords[event.inst.subarray.tel_indices[tel_id]]
    distance = np.sqrt((event.mc.core_x - tel_pos.x)**2 + (event.mc.core_y - tel_pos.y)**2)
    return distance

In [ ]:
source = event_source(infile, max_events=max_events)

image_container = Image()
dl1_parameters = DL1_parameters()
dl1_parameters.prefix = ''

output_filename = outdir + '/DL1_integrations.h5'

In [ ]:
for integrator in integrators:
    print(integrator)    
    with HDF5TableWriter(output_filename, group_name=integrator, mode='a', add_prefix=True) as h5_table:

        cal = CameraCalibrator(r1_product='HESSIOR1Calibrator',
                           extractor_product=integrator)

        for event in source:
            cal.calibrate(event)
            
            for tel_id in event.r0.tels_with_data:
                tel = event.inst.subarray.tel[tel_id]
                cam_id = tel.camera.cam_id
                
                dl1_parameters.mc_distance_tel_impact = tel_to_impact_point_distance(event, tel_id)

                image_container.calibrated = event.dl1.tel[tel_id].image
                image_container.true_pe = event.mc.tel[tel_id].photo_electron_image
                
                h5_table.write(cam_id, [image_container, event.mc, event.r0, dl1_parameters])
